In [1]:
import pandas as pd
#from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
import joblib
import hopsworks


In [2]:

project = hopsworks.login()

fs = project.get_feature_store()

2025-04-23 19:25:08,033 INFO: Initializing external client
2025-04-23 19:25:08,036 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-04-23 19:25:08,558 WARNING: UserWarning: The installed hopsworks client version 4.1.8 may not be compatible with the connected Hopsworks backend version 4.2.0. 
To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'



2025-04-23 19:25:09,300 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1220788


In [19]:
users_fg = fs.get_feature_group(
    name="users",
    version=1
)

events_fg = fs.get_feature_group(
    name="events",
    version=1
)

weather_rank_fg = fs.get_feature_group(
    name="weather_ranking",
    version=1
)

no_weather_rank_fg = fs.get_feature_group(
    name="no_weather_ranking",
    version=1
)

## <span style="color:#ff5f27">⚙️ Feature View Creation </span>

In [17]:
# Select features
selected_features_customers = users_fg.select_all()

fs.get_or_create_feature_view( 
    name='users',
    query=selected_features_customers,
    version=1,
)

In [18]:
# Select features
selected_features_articles = events_fg.select_all()

fs.get_or_create_feature_view(
    name='events',
    query=selected_features_articles,
    version=1,
)

In [25]:
NO_WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
       'event_type','event_city', 'duration',
       'attendance_rate', 'event_indoor_capability', 'user_city',
       'age', 'user_interests','label']

WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
       'event_type','event_city', 'duration','weather_condition', 'temperature',
       'attendance_rate', 'event_indoor_capability', 'user_city',
       'user_weather_preference', 'age', 'user_interests','label']

In [22]:
# Select features
features_weather_ranking = weather_rank_fg.select(WEATHER_SELECTED_FEATURES)

feature_view_ranking_weather = fs.get_or_create_feature_view(
    name='weather_ranking_2',
    query=features_weather_ranking,
    labels=["label"],
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/weather_ranking_2/version/1


In [26]:
# Select features
features_no_weather_ranking = no_weather_rank_fg.select(NO_WEATHER_SELECTED_FEATURES)

feature_view_ranking_no_weather = fs.get_or_create_feature_view(
    name='no_weather_ranking_2',
    query=features_no_weather_ranking,
    labels=["label"],
    version=1,
)

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1220788/fs/1208418/fv/no_weather_ranking_2/version/1


---

---

---

---

In [27]:
# Get feature views
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking_2', version=1)


In [28]:
# Get feature views
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking_2', version=1)

feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking_2', version=1)

In [29]:
# Get feature views
feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking_2', version=1)

feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking_2', version=1)
# Get training and validation data directly from feature views
weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
    feature_view_ranking_weather.train_test_split(
    test_size=0.1,
    description='Weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.67s) 
2025-04-23 19:56:07,750 WARNING: VersionWarning: Incremented version to `1`.



In [54]:
weather_X_train.head()

,interaction_distance_to_event,title,event_type,event_city,duration,weather_condition,temperature,attendance_rate,event_indoor_capability,user_city,user_weather_preference,age,user_interests
1,16921.000000,synchronized solutionoriented migration commun...,Community & Causes,Paris,240,Cloudy,16.9,79.345494,False,Sydney,outdoor,46,travel
2,10.584158,cloned valueadded graphical user interface hea...,Health & Wellness,Paris,240,Clear,18.1,12.692646,False,Paris,outdoor,49,food
3,449.000000,reverseengineered tertiary neuralnet entertain...,Entertainment,Toronto,240,Rain,9.1,83.238042,True,New York,any,32,cinema tech music sports
4,7.000000,objectbased local support technology in new york,Technology,New York,240,Cloudy,16.0,81.857555,True,New York,indoor,34,tech literature
5,6581.000000,publickey intermediate budgetary management sp...,Sports & Fitness,Berlin,180,Cloudy,13.3,76.052787,False,Toronto,outdoor,34,food fashion


In [31]:

no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
    feature_view_ranking_no_weather.train_test_split(
    test_size=0.1,
    description='No-weather ranking training dataset',
)


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.86s) 
2025-04-23 19:57:36,700 WARNING: VersionWarning: Incremented version to `1`.



In [32]:
weather_X_train.columns


Index(['interaction_distance_to_event', 'title', 'event_type', 'event_city',
       'duration', 'weather_condition', 'temperature', 'attendance_rate',
       'event_indoor_capability', 'user_city', 'user_weather_preference',
       'age', 'user_interests'],
      dtype='object')

In [14]:
NO_WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
       'event_type','event_city', 'duration',
       'attendance_rate', 'event_indoor_capability', 'user_city',
       'age', 'user_interests']

WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
       'event_type','event_city', 'duration','weather_condition', 'temperature',
       'attendance_rate', 'event_indoor_capability', 'user_city',
       'user_weather_preference', 'age', 'user_interests']

In [2]:
import pandas as pd
users_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/users.csv')
events_df = pd.read_csv("/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/events.csv")
interactions_df = pd.read_csv('/home/nkama/masters_thesis_project/thesis/partially_synthetic/data/main_data/interactions.csv')



# Merge user/event features into interactions
interactions_df = interactions_df.merge(users_df, on="user_id")
interactions_df = interactions_df.merge(events_df, on="event_id", suffixes=('_user', '_event'))
from sklearn.model_selection import train_test_split

NO_WEATHER_SELECTED_FEATURES =['interaction_type',
       'distance_to_event', 'interaction_label',
        'gender', 'joinedAt', 'location', 'age',
      'indoor_outdoor_preference', 'user_interests', 
       'start_time', 'city', 'yes_count',
       'maybe_count', 'invited_count', 'no_count', 'total_users', 'category', 
       'title', 'event_type','event_indoor_capability']

WEATHER_SELECTED_FEATURES =['interaction_type',
       'distance_to_event', 'interaction_label',
        'gender', 'joinedAt', 'location', 'age',
      'indoor_outdoor_preference',
       'start_time', 'city', 'yes_count',
       'maybe_count', 'invited_count', 'no_count', 'total_users',
       'weather_description', 'category','event_type',
       'event_indoor_capability', 'temperature_2m_mean', 'precipitation_sum']

# )
# Splitting the dataset into features and labels
weather_X = interactions_df[WEATHER_SELECTED_FEATURES]  # Features
weather_y = interactions_df['interaction_label']   

no_weather_X = interactions_df[NO_WEATHER_SELECTED_FEATURES]  # Features
no_weather_y = interactions_df['interaction_label']                   # Labels

# Splitting the dataset into training and evaluation sets
weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
    train_test_split(weather_X, weather_y, test_size=0.2, random_state=42)

no_weather_X_train, no_weather_X_val, no_weather_y_train, no_weather_y_val = \
    train_test_split(no_weather_X, no_weather_y, test_size=0.2, random_state=42)

In [3]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
# Get feature views
# feature_view_ranking_weather = fs.get_feature_view(name='weather_ranking_2', version=1)

# feature_view_ranking_no_weather = fs.get_feature_view(name='no_weather_ranking_2', version=1)
# # Get training and validation data directly from feature views
# weather_X_train, weather_X_val, weather_y_train, weather_y_val = \
#     feature_view_ranking_weather.train_test_split(
#     test_size=0.1,
#     description='Weather ranking training dataset',
# )

# NO_WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
#        'event_type','event_city', 'duration',
#        'attendance_rate', 'event_indoor_capability', 'user_city',
#        'age', 'user_interests']

# WEATHER_SELECTED_FEATURES =['interaction_distance_to_event', 'title',
#        'event_type','event_city', 'duration','weather_condition', 'temperature',
#        'attendance_rate', 'event_indoor_capability', 'user_city',
#        'user_weather_preference', 'age', 'user_interests']

# Final version without text fields (title, user_interests)
def train_catboost_without_text_fields(
    train_df, val_df, train_y, val_y
):
    # Drop the text fields if present
    text_columns = ["title", "user_interests"]
    train_df = train_df.drop(columns=[col for col in text_columns if col in train_df.columns])
    val_df = val_df.drop(columns=[col for col in text_columns if col in val_df.columns])

    # Identify categorical features
    cat_features = train_df.select_dtypes(include=["object", "bool"]).columns.tolist()

    # Create CatBoost Pools
    train_pool = Pool(train_df, train_y, cat_features=cat_features)
    val_pool = Pool(val_df, val_y, cat_features=cat_features)

    # Train the model
    model = CatBoostClassifier(
        learning_rate=0.2,
        iterations=100,
        depth=10,
        early_stopping_rounds=5,
        use_best_model=True,
        scale_pos_weight=None, 
        verbose=False
    )


    model.fit(train_pool, eval_set=val_pool)

    # Evaluation
    preds = model.predict(val_pool)
    precision, recall, fscore, _ = precision_recall_fscore_support(val_y, preds, average="binary")
    print("\nClassification Report:")
    print(classification_report(val_y, preds))

    metrics = {
        "precision": precision,
        "recall": recall,
        "fscore": fscore,
    }
    from sklearn.metrics import confusion_matrix
    import numpy as np

    preds = model.scores = model.predict_proba(val_pool)[:, 1] 
    print("Predicted Class Distribution:", np.unique(preds, return_counts=True))

    # print("\nConfusion Matrix:")
    # print(confusion_matrix(val_y, preds))

    return model, metrics, val_pool

"✅ Updated CatBoost training function to exclude title and user_interests for memory safety."


'✅ Updated CatBoost training function to exclude title and user_interests for memory safety.'

In [13]:
# ... existing code ...
numeric_columns = weather_X_train.select_dtypes(include=['number']).columns.tolist()
print(numeric_columns)
# ... existing code ...

['interaction_distance_to_event', 'event_lat', 'event_lon', 'duration', 'temperature', 'attendance_rate', 'user_lat', 'user_lon', 'age']


In [35]:
# ... existing code ...
categorical_columns = no_weather_X_train.select_dtypes(include=['object', 'category']).columns.tolist()
print(categorical_columns)
# ... existing code ...

['title', 'event_type', 'event_city', 'user_city', 'user_interests']


In [48]:
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import classification_report, precision_recall_fscore_support
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from scipy.sparse import hstack
import numpy as np
import pandas as pd

def train_catboost_with_text_fields(train_df, val_df, train_y, val_y):
    # Step 1: Process text fields with TF-IDF
    tfidf_title = TfidfVectorizer(max_features=50)
    tfidf_interests = TfidfVectorizer(max_features=50)
    
    # Fit and transform text features
    X_train_text = hstack([
        tfidf_title.fit_transform(train_df["title"].fillna("")),
        tfidf_interests.fit_transform(train_df["user_interests"].fillna(""))
    ])
    X_val_text = hstack([
        tfidf_title.transform(val_df["title"].fillna("")),
        tfidf_interests.transform(val_df["user_interests"].fillna(""))
    ])

    # Step 2: Process numeric features
    numeric_cols = train_df.select_dtypes(include=['number']).columns.tolist()
    train_df[numeric_cols] = train_df[numeric_cols].fillna(0)
    val_df[numeric_cols] = val_df[numeric_cols].fillna(0)
    
    scaler = StandardScaler().fit(train_df[numeric_cols])
    X_train_numeric = scaler.transform(train_df[numeric_cols])
    X_val_numeric = scaler.transform(val_df[numeric_cols])

    # Step 3: Process categorical features
    categorical_cols = train_df.select_dtypes(include=['object', 'category']).columns.tolist()
    categorical_cols = [col for col in categorical_cols if col not in ["title", "user_interests"]]
    
    # Fill missing values
    for col in categorical_cols:
        train_df[col] = train_df[col].fillna('unknown')
        val_df[col] = val_df[col].fillna('unknown')
    
    # Extract categorical features as separate DataFrames
    X_train_cat = train_df[categorical_cols].reset_index(drop=True)
    X_val_cat = val_df[categorical_cols].reset_index(drop=True)
    
    # Convert TF-IDF and numeric features to numpy arrays
    X_train_text_numeric = hstack([X_train_text, X_train_numeric]).toarray()
    X_val_text_numeric = hstack([X_val_text, X_val_numeric]).toarray()
    
    # Create DataFrames for the text and numeric features
    X_train_text_numeric_df = pd.DataFrame(
        X_train_text_numeric, 
        columns=[f'tfidf_feature_{i}' for i in range(X_train_text_numeric.shape[1])]
    )
    X_val_text_numeric_df = pd.DataFrame(
        X_val_text_numeric, 
        columns=[f'tfidf_feature_{i}' for i in range(X_val_text_numeric.shape[1])]
    )
    
    # Combine all features into DataFrames
    X_train_all = pd.concat([X_train_text_numeric_df, X_train_cat], axis=1)
    X_val_all = pd.concat([X_val_text_numeric_df, X_val_cat], axis=1)
    
    # Define categorical feature indices (positions in the final DataFrame)
    cat_features = list(range(X_train_text_numeric.shape[1], X_train_all.shape[1]))
    
    # Create Pool objects with proper categorical features
    train_pool = Pool(data=X_train_all, label=train_y, cat_features=cat_features)
    val_pool = Pool(data=X_val_all, label=val_y, cat_features=cat_features)

    # Train the model with CatBoost-specific parameters
    model = CatBoostClassifier(
        learning_rate=0.2,
        iterations=100,
        depth=10,
        early_stopping_rounds=5,
        use_best_model=True,
        verbose=False
    )

    model.fit(train_pool, eval_set=val_pool)

    # Evaluation
    preds = model.predict(val_pool)
    precision, recall, fscore, _ = precision_recall_fscore_support(val_y, preds, average="binary")
    print("\nClassification Report:")
    print(classification_report(val_y, preds))

    # Store probabilities for later use
    model.scores = model.predict_proba(val_pool)[:, 1]
    
    metrics = {
        "precision": precision,
        "recall": recall,
        "fscore": fscore,
    }
    
    return model, metrics, val_pool


In [4]:
# Use this function to train on your weather / no-weather datasets
weather_model, weather_metrics, weather_val_pool = train_catboost_without_text_fields(
    train_df=weather_X_train,
    val_df=weather_X_val,
    train_y=weather_y_train,
    val_y=weather_y_val
)

# Save the models using Joblib
joblib.dump(weather_model, 'weather_ranking_model.pkl')
print("\nModels saved successfully!")


Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       325
         1.0       1.00      1.00      1.00      5325

    accuracy                           1.00      5650
   macro avg       1.00      1.00      1.00      5650
weighted avg       1.00      1.00      1.00      5650

Predicted Class Distribution: (array([2.41121643e-04, 2.49152161e-04, 2.49167986e-04, ...,
       9.99983900e-01, 9.99983901e-01, 9.99983902e-01]), array([1, 1, 1, ..., 1, 1, 1]))


NameError: name 'joblib' is not defined

In [5]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        weather_X_train.columns, 
        weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_type': 76.5778113641695,
 'weather_description': 4.30674008101765,
 'age': 3.237467817179253,
 'category': 2.9370056032853573,
 'maybe_count': 2.7196625311286935,
 'event_indoor_capability': 2.2254651728353663,
 'indoor_outdoor_preference': 2.1986309453821233,
 'temperature_2m_mean': 1.9819653541549123,
 'distance_to_event': 1.712327496290426,
 'total_users': 0.8689565646688637,
 'yes_count': 0.7116664731004836,
 'invited_count': 0.3244445822618073,
 'precipitation_sum': 0.14332783183238257,
 'interaction_label': 0.05452766519541397,
 'no_count': 2.939750863428235e-07,
 'gender': 2.2352268228832082e-07,
 'joinedAt': 0.0,
 'location': 0.0,
 'start_time': 0.0,
 'city': 0.0,
 'event_type': 0.0}

In [6]:
# Use this function to train on your weather / no-weather datasets
no_weather_model, no_weather_metrics, no_weather_val_pool = train_catboost_without_text_fields(
    train_df=no_weather_X_train,
    val_df=no_weather_X_val,
    train_y=no_weather_y_train,
    val_y=no_weather_y_val
)

joblib.dump(no_weather_model, 'no_weather_ranking_model.pkl')
print("\nModels saved successfully!")


Classification Report:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       325
         1.0       1.00      1.00      1.00      5325

    accuracy                           1.00      5650
   macro avg       1.00      1.00      1.00      5650
weighted avg       1.00      1.00      1.00      5650

Predicted Class Distribution: (array([4.23551057e-04, 4.30403995e-04, 4.39265302e-04, ...,
       9.99971194e-01, 9.99971230e-01, 9.99971461e-01]), array([1, 1, 1, ..., 1, 1, 2]))


NameError: name 'joblib' is not defined

In [28]:
# Use this function to train on your weather / no-weather datasets
no_weather_model, no_weather_metrics, pool_no_weather_val = train_catboost_without_text_fields(
    train_df=no_weather_X_train,
    val_df=no_weather_X_val,
    train_y=no_weather_y_train,
    val_y=no_weather_y_val
)


Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.81      0.75      8951
           1       0.72      0.58      0.64      7414

    accuracy                           0.71     16365
   macro avg       0.71      0.70      0.70     16365
weighted avg       0.71      0.71      0.70     16365

Predicted Class Distribution: (array([0.25162631, 0.25221187, 0.25478915, ..., 0.83534972, 0.83585689,
       0.83656814]), array([1, 1, 1, ..., 1, 1, 1]))


In [7]:
feat_to_score = {
    feature: score 
    for feature, score 
    in zip(
        no_weather_X_train.columns, 
        no_weather_model.feature_importances_,
    )
}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True,
    )
)
feat_to_score

{'interaction_type': 54.72122052353688,
 'interaction_label': 30.8715625371324,
 'maybe_count': 2.3459777055818587,
 'city': 2.2449991247582415,
 'no_count': 1.6503214458250421,
 'age': 1.5090039137423563,
 'total_users': 1.4746309724073807,
 'distance_to_event': 1.1914178305045156,
 'yes_count': 1.1667068796907545,
 'indoor_outdoor_preference': 1.1514013475146947,
 'category': 0.7191426447743281,
 'title': 0.48369153655103775,
 'invited_count': 0.3897249471402347,
 'start_time': 0.08019859084027804,
 'gender': 0.0,
 'joinedAt': 0.0,
 'location': 0.0,
 'user_interests': 0.0}

In [8]:
from sklearn.metrics import roc_auc_score, average_precision_score, ndcg_score, precision_score, recall_score
import numpy as np


def evaluate_ranking_model_proba(model, val_pool, val_y, k_list=[5, 10]):
    """
    Evaluate a CatBoost ranking model using predicted probabilities, not binary class outputs.
    """

    # Predict class probabilities (not class labels)
    proba = model.predict_proba(val_pool)[:, 1]  # Probability for class 1

    results = {
        "AUC": roc_auc_score(val_y, proba),
        "Average Precision (MAP)": average_precision_score(val_y, proba),
    }

    # Convert to numpy arrays
    true_labels = np.array(val_y)
    predicted_scores = np.array(proba)

    # Sort by predicted score
    sorted_indices = np.argsort(predicted_scores)[::-1]
    sorted_true = true_labels[sorted_indices]

    for k in k_list:
        top_k = sorted_true[:k]
        precision_at_k = np.mean(top_k)
        recall_at_k = np.sum(top_k) / np.sum(true_labels)
        ndcg_at_k = ndcg_score(
            y_true=true_labels.reshape(1, -1),
            y_score=predicted_scores.reshape(1, -1),
            k=k
        )



        results[f"Precision@{k}"] = precision_at_k
        results[f"Recall@{k}"] = recall_at_k
        results[f"NDCG@{k}"] = ndcg_at_k

    return results

"✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K."


'✅ Evaluation function ready: scores ranking model using AUC, MAP, Precision@K, Recall@K, and NDCG@K.'

In [9]:

# Evaluate weather-aware model
weather_scores = evaluate_ranking_model_proba(
    model=weather_model,
    val_pool=weather_val_pool,
    val_y=weather_y_val
)

# Evaluate no-weather model
no_weather_scores = evaluate_ranking_model_proba(
    model=no_weather_model,
    val_pool=no_weather_val_pool,
    val_y=no_weather_y_val
)

# Compare results
print("Weather Model Scores:")
for k, v in weather_scores.items():
    print(f"{k}: {v:.4f}")

print("\nNo-Weather Model Scores:")
for k, v in no_weather_scores.items():
    print(f"{k}: {v:.4f}")


Weather Model Scores:
AUC: 1.0000
Average Precision (MAP): 1.0000
Precision@5: 1.0000
Recall@5: 0.0009
NDCG@5: 1.0000
Precision@10: 1.0000
Recall@10: 0.0019
NDCG@10: 1.0000

No-Weather Model Scores:
AUC: 1.0000
Average Precision (MAP): 1.0000
Precision@5: 1.0000
Recall@5: 0.0009
NDCG@5: 1.0000
Precision@10: 1.0000
Recall@10: 0.0019
NDCG@10: 1.0000


In [ ]:
import pickle
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import roc_curve, auc, confusion_matrix, accuracy_score
import seaborn as sns
from datetime import datetime

def evaluate_and_save_model(model, val_pool, val_y, model_name="catboost_model", ranking_dir="ranking_models"):
    """
    Evaluate the CatBoost model and save it for future use
    
    Parameters:
    -----------
    model : CatBoostClassifier
        Trained CatBoost model
    val_pool : Pool
        Validation data pool
    val_y : array-like
        Validation labels
    model_name : str
        Name for the saved model
    save_dir : str
        Directory to save the model and evaluation results
    
    Returns:
    --------
    dict
        Dictionary containing evaluation metrics
    """
    # Create directory if it doesn't exist
    if not os.path.exists(ranking_dir):
        os.makedirs(ranking_dir)
    
    # Generate timestamp for model versioning
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"{model_name}_{timestamp}"
    
    # 1. Basic evaluation metrics
    y_pred = model.predict(val_pool)
    y_pred_proba = model.predict_proba(val_pool)[:, 1]
    
    accuracy = accuracy_score(val_y, y_pred)
    
    # 2. ROC Curve and AUC
    fpr, tpr, _ = roc_curve(val_y, y_pred_proba)
    roc_auc = auc(fpr, tpr)
    
    # Plot ROC curve
    plt.figure(figsize=(10, 8))
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC)')
    plt.legend(loc="lower right")
    plt.savefig(f"{ranking_dir}/{model_filename}_roc_curve.png")
    
    # 3. Confusion Matrix
    plt.figure(figsize=(8, 6))
    cm = confusion_matrix(val_y, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.title('Confusion Matrix')
    plt.savefig(f"{ranking_dir}/{model_filename}_confusion_matrix.png")
    
    # 4. Feature Importance
    plt.figure(figsize=(12, 8))
    feature_importances = model.get_feature_importance(val_pool)
    feature_names = model.feature_names_
    
    # Sort feature importances
    indices = np.argsort(feature_importances)[-20:]  # top 20 features
    plt.barh(range(len(indices)), feature_importances[indices])
    plt.yticks(range(len(indices)), [feature_names[i] for i in indices])
    plt.xlabel('Feature Importance')
    plt.title('Top 20 Feature Importances')
    plt.tight_layout()
    plt.savefig(f"{ranking_dir}/{model_filename}_feature_importance.png")
    
    # 5. Save model and performance metrics
    # Save the trained model
    with open(f"{ranking_dir}/{model_filename}.pkl", "wb") as f:
        pickle.dump(model, f)
    
    # Save evaluation metrics
    metrics = {
        "accuracy": accuracy,
        "roc_auc": roc_auc,
        "confusion_matrix": cm.tolist(),
        "feature_importance": dict(zip(feature_names, feature_importances)),
        "model_path": f"{ranking_dir}/{model_filename}.pkl"
    }
    
    # Save metrics to JSON file
    import json
    with open(f"{ranking_dir}/{model_filename}_metrics.json", "w") as f:
        json.dump(metrics, f, indent=4)
    
    print(f"Model saved to {ranking_dir}/{model_filename}.pkl")
    print(f"Evaluation metrics saved to {ranking_dir}/{model_filename}_metrics.json")
    print(f"ROC AUC: {roc_auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    
    return metrics



In [ ]:


# Assuming weather_model is your trained model from previous code
metrics = evaluate_and_save_model(
    model=weather_model,
    val_pool=weather_val_pool,  # This should be the validation Pool object created earlier
    val_y=weather_y_val,
    model_name="weather_classifier",
    save_dir="weather_models"
)


In [ ]:

# If you need to load the model later
def load_model(model_path):
    """Load a saved CatBoost model"""
    with open(model_path, "rb") as f:
        model = pickle.load(f)
    return model

# Example of using the saved model for prediction with new data
def predict_with_saved_model(model_path, new_data):
    """
    Make predictions using a saved model
    
    Parameters:
    -----------
    model_path : str
        Path to the saved model file
    new_data : Pool or DataFrame
        New data to make predictions on
        
    Returns:
    --------
    array
        Predicted probabilities
    """
    model = load_model(model_path)
    
    # If new_data is not already a Pool object, you'll need to convert it
    # following the same preprocessing steps as during training
    
    # Return predictions
    return model.predict_proba(new_data)[:, 1]

In [34]:
# Connect to Hopsworks Model Registry
model_registry = project.get_model_registry()

In [35]:


from hsml.schema import Schema
from hsml.model_schema import ModelSchema
import joblib

def upload_catboost_model_to_hopsworks(
    model, model_name: str, model_description: str,
    model_registry, X_train, y_train, metrics: dict,
    local_path
):
    # Save the model locally
    joblib.dump(model, local_path)

    # Create input/output schema
    input_schema = Schema(X_train)
    output_schema = Schema(y_train)
    model_schema = ModelSchema(input_schema, output_schema)

    # Sample input for UI representation
    input_example = X_train.sample().to_dict("records")

    # Create and save model in Hopsworks
    hopsworks_model = model_registry.python.create_model(
        name=model_name,
        metrics=metrics,
        model_schema=model_schema,
        input_example=input_example,
        description=model_description,
    )

    hopsworks_model.save(local_path)

    print(f"✅ Model '{model_name}' successfully uploaded to Hopsworks.")
    return hopsworks_model

"✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference."


'✅ Model upload function is ready for Hopsworks. Pass your CatBoost model, train data, metrics, and registry reference.'

In [36]:
upload_catboost_model_to_hopsworks(
    model=weather_model,
    model_name="weather_ranking_model",
    model_description="Ranking model trained with weather features",
    model_registry=project.get_model_registry(),
    X_train=weather_X_train, 
    y_train=weather_y_train,
    metrics=weather_metrics,
    local_path="weather_ranking_model.pkl"
)


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/262968 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/598 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1586 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/weather_ranking_model/2
✅ Model 'weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'weather_ranking_model', version: 2)

In [37]:
upload_catboost_model_to_hopsworks(
    model=no_weather_model,
    model_name="no_weather_ranking_model",
    model_description="Ranking model trained without weather features",
    model_registry=project.get_model_registry(),
    X_train=no_weather_X_train, 
    y_train=no_weather_y_train,
    metrics=no_weather_metrics,
    local_path="no_weather_ranking_model.pkl"
)

  0%|          | 0/6 [00:00<?, ?it/s]

Uploading: 0.000%|          | 0/901551 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/505 elapsed<00:00 remaining<?

Uploading: 0.000%|          | 0/1348 elapsed<00:00 remaining<?

Model created, explore it at https://c.app.hopsworks.ai:443/p/1220788/models/no_weather_ranking_model/2
✅ Model 'no_weather_ranking_model' successfully uploaded to Hopsworks.


Model(name: 'no_weather_ranking_model', version: 2)

In [19]:
import numpy as np
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from geopy.distance import geodesic
from scipy.stats import skewnorm, zipf
from mimesis import Generic
import re

# Initialize components
fake = Faker()
generic = Generic('en')
np.random.seed(42)
random.seed(42)

# ====================== #
# 1. Core Configuration #
# ====================== #
cities = ['New York', 'London', 'Paris', 'Tokyo', 'Sydney', 
          'Berlin', 'Mumbai', 'São Paulo', 'Toronto', 'Dubai']
city_probs = [0.2, 0.15, 0.1, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.05]
city_coords = {
    k: (float(v[0]), float(v[1])) for k,v in {
        'New York': (40.7128, -74.0060), 'London': (51.5074, -0.1278),
        'Paris': (48.8566, 2.3522), 'Tokyo': (35.6762, 139.6503),
        'Sydney': (-33.8688, 151.2093), 'Berlin': (52.5200, 13.4050),
        'Mumbai': (19.0760, 72.8777), 'São Paulo': (-23.5505, -46.6333),
        'Toronto': (43.6532, -79.3832), 'Dubai': (25.2048, 55.2708)
    }.items()
}

# ====================== #
# 2. Helper Functions #
# ====================== #
def generate_location(city):
    base_lat, base_lon = city_coords[city]
    return (
        base_lat + np.random.uniform(-0.1, 0.1),
        base_lon + np.random.uniform(-0.1, 0.1)
    )

def get_relevant_events(user, events):
    # Access user_interests as attribute, not dictionary key
    user_interests = set(user.user_interests.split(','))
    return events[
        events.event_type.apply(
            lambda x: any(i.lower() in x.lower() for i in user_interests)
        )
    ]


def safe_sample(df, n):
    return df.sample(n, replace=True) if len(df) > 0 else pd.DataFrame()

# ====================== #
# 3. User Generation #
# ====================== #
def generate_users(n_users=20000):
    users = []
    interests = ['music', 'sports', 'tech', 'food', 'art', 
                'literature', 'cinema', 'travel', 'fitness', 'fashion']
    
    for _ in range(n_users):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        age = max(18, min(100, int(skewnorm.rvs(5, loc=25, scale=15))))
        
        # Generate interests with power-law distribution
        num_interests = min(zipf.rvs(1.2), 4)
        user_interests = random.sample(interests, k=num_interests)
        
        users.append({
            'user_id': generic.person.identifier(mask='@@###@'),
            'user_lat': lat,
            'user_lon': lon,
            'user_city': city,
            'age': age,
            'user_interests': ','.join(user_interests),
            'social_connectedness': np.random.poisson(lam=15)
        })
    
    return pd.DataFrame(users)

# ====================== #
# 4. Event Generation #
# ====================== #
def generate_events(n_events=5000):
    events = []
    event_types = [
        'Education & Learning', 'Technology', 'Seasonal & Festivals', 
        'Arts & Culture', 'Entertainment', 'Sports & Fitness', 
        'Business & Networking', 'Health & Wellness', 'Music & Concerts', 
        'Food & Drink', 'Community & Causes', 'Immersive Experiences'
    ]
    
    for _ in range(n_events):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        start_time = fake.date_time_between(
            start_date=datetime(2025, 3, 27), 
            end_date=datetime(2025, 12, 31)
        )
        
        events.append({
            'event_id': generic.person.identifier(mask='@@###@'),
            'title': f"{fake.catch_phrase()} in {city}",
            'event_type': random.choice(event_types),
            'event_lat': lat,
            'event_lon': lon,
            'event_city': city,
            'start_time': start_time,
            'duration': random.choice([120, 180, 240]),
            'expected_attendance': int(np.random.lognormal(mean=6, sigma=0.5))
        })
    
    return pd.DataFrame(events)

# ====================== #
# 5. Interaction Generation #
# ====================== #
def generate_interactions(users, events, n_interactions=150000):
    interactions = []
    
    # Phase 1: Base users (attend few events)
    base_users = users.sample(frac=0.2)
    for user in base_users.itertuples():
        num_events = min(zipf.rvs(2.5), 2)  # ⬅ lower number of events
        candidate_events = get_relevant_events(user, events)
        selected_events = safe_sample(candidate_events, num_events)
        for event in selected_events.itertuples():
            interactions.append(create_interaction(user, event))

    # Phase 2: Regular users
    regular_users = users.drop(base_users.index).sample(frac=0.3)
    for user in regular_users.itertuples():
        num_events = min(zipf.rvs(1.5) + 4, 12)  # ⬅ still modest
        candidate_events = get_relevant_events(user, events)
        selected_events = safe_sample(candidate_events, num_events)
        for event in selected_events.itertuples():
            interactions.append(create_interaction(user, event))

    # Phase 3: Power users
    power_users = users.nlargest(int(len(users)*0.5), 'social_connectedness')
    for user in power_users.itertuples():
        num_events = min(zipf.rvs(1.2) + 10, 40)  # ⬅ significantly higher
        candidate_events = events.sample(250)
        selected_events = safe_sample(candidate_events, num_events)
        for event in selected_events.itertuples():
            interactions.append(create_interaction(user, event))

    # Final interaction DataFrame
    df = pd.DataFrame(interactions)
    df = df.sample(frac=1).reset_index(drop=True)

    print("Interaction distribution by user:")
    print(df['user_id'].value_counts().value_counts().head(10))  # Debug: see how many users attend how many events
    
    return df

    #return balance_labels(df)

def create_interaction(user, event):
    distance = geodesic(
        (user.user_lat, user.user_lon),  # Changed from ['user_lat']
        (event.event_lat, event.event_lon)
    ).km
    
    interest_match = len(
        set(user.user_interests.split(',')) &  # Changed from ['user_interests']
        set(event.event_type.split(' & '))
    ) > 0
    
    return {
        'user_id': user.user_id,  # Changed from ['user_id']
        'event_id': event.event_id,
        'distance_to_event': distance,
        'event_type_in_user_interests': int(interest_match),
        'interest_match': interest_match,
        'interaction_label': 1 if random.random() < (0.6 if interest_match else 0.4) else 0
    }


def balance_labels(df):
    pos = df[df.interaction_label == 1]
    neg = df[df.interaction_label == 0]
    min_count = min(len(pos), len(neg))
    
    return pd.concat([
        pos.sample(min_count, replace=True),
        neg.sample(min_count, replace=True)
    ]).sample(frac=1).reset_index(drop=True)

# ====================== #
# 6. Execution #
# ====================== #
if __name__ == "__main__":
    users_df = generate_users(10000)
    events_df = generate_events(5000)
    interactions_df = generate_interactions(users_df, events_df)
    
    # # Save datasets
    # users_df.to_csv('synthetic_users.csv', index=False)
    # events_df.to_csv('synthetic_events.csv', index=False)
    # interactions_df.to_csv('synthetic_interactions.csv', index=False)


Interaction distribution by user:
count
40    1367
1      707
12     629
11     614
5      429
41     292
13     234
2      224
45     208
16     158
Name: count, dtype: int64


In [16]:
len(interactions_df)

107309

In [21]:
events_per_user = interactions_df.groupby("user_id").size()
max(events_per_user)

for i in range(1, 50):
    print(f"{events_per_user[events_per_user == i].size} attended {i} events")

707 attended 1 events
224 attended 2 events
0 attended 3 events
0 attended 4 events
429 attended 5 events
153 attended 6 events
65 attended 7 events
55 attended 8 events
30 attended 9 events
20 attended 10 events
614 attended 11 events
629 attended 12 events
234 attended 13 events
146 attended 14 events
102 attended 15 events
158 attended 16 events
129 attended 17 events
121 attended 18 events
71 attended 19 events
79 attended 20 events
68 attended 21 events
54 attended 22 events
106 attended 23 events
74 attended 24 events
56 attended 25 events
50 attended 26 events
49 attended 27 events
31 attended 28 events
29 attended 29 events
31 attended 30 events
24 attended 31 events
23 attended 32 events
24 attended 33 events
21 attended 34 events
22 attended 35 events
17 attended 36 events
31 attended 37 events
26 attended 38 events
12 attended 39 events
1367 attended 40 events
292 attended 41 events
109 attended 42 events
0 attended 43 events
6 attended 44 events
208 attended 45 events
66 at

In [22]:

interactions_df["interaction_label"].value_counts()

interaction_label
0    91978
1    61130
Name: count, dtype: int64

In [23]:
users_df.columns

Index(['user_id', 'user_lat', 'user_lon', 'user_city', 'age', 'user_interests',
       'social_connectedness'],
      dtype='object')

In [24]:
events_df.columns

Index(['event_id', 'title', 'event_type', 'event_lat', 'event_lon',
       'event_city', 'start_time', 'duration', 'expected_attendance'],
      dtype='object')

In [25]:
interactions_df.columns

Index(['user_id', 'event_id', 'distance_to_event',
       'event_type_in_user_interests', 'interest_match', 'interaction_label'],
      dtype='object')

In [26]:
interactions_df.head()

,user_id,event_id,distance_to_event,event_type_in_user_interests,interest_match,interaction_label
0,JE120K,AK136W,9601.453066,0,False,0
1,DV609A,MH837R,6289.333916,0,False,0
2,EF903S,UL464S,540.419648,0,False,0
3,UF178B,LD259A,11088.821115,0,False,0
4,DF768O,TS601C,8.373094,0,False,0


In [27]:
import numpy as np
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from geopy.distance import geodesic
from scipy.stats import skewnorm, dirichlet
from mimesis import Generic
import re

# Initialize Faker and set seed for reproducibility
fake = Faker()
generic = Generic('en')
np.random.seed(42)
random.seed(42)

# City configurations
cities = ['New York', 'London', 'Paris', 'Tokyo', 'Sydney', 
          'Berlin', 'Mumbai', 'São Paulo', 'Toronto', 'Dubai']
city_probs = [0.2, 0.15, 0.1, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.05]
city_coords = {
    k: (float(v[0]), float(v[1])) for k,v in {
        'New York': (40.7128, -74.0060), 'London': (51.5074, -0.1278),
        'Paris': (48.8566, 2.3522), 'Tokyo': (35.6762, 139.6503),
        'Sydney': (-33.8688, 151.2093), 'Berlin': (52.5200, 13.4050),
        'Mumbai': (19.0760, 72.8777), 'São Paulo': (-23.5505, -46.6333),
        'Toronto': (43.6532, -79.3832), 'Dubai': (25.2048, 55.2708)
    }.items()
}

def generate_location(city):
    base_lat, base_lon = city_coords[city]
    if random.random() < 0.8:
        lat = base_lat + np.random.uniform(-0.1, 0.1)
        lon = base_lon + np.random.uniform(-0.1, 0.1)
    else:
        lat = base_lat + np.random.uniform(-2, 2)
        lon = base_lon + np.random.uniform(-2, 2)
    return lat, lon

def generate_users(n_users=20000):
    users = []
    interests = ['music', 'sports', 'tech', 'food', 'art', 
                'literature', 'cinema', 'travel', 'fitness', 'fashion']
    
    for _ in range(n_users):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        age = max(18, min(100, int(skewnorm.rvs(5, loc=25, scale=15))))
        weather_probs = dirichlet.rvs([0.3, 0.5, 0.2])[0]
        
        # Generate interests with power-law distribution
        num_interests = min(np.random.zipf(1.2), 5)
        user_interests = random.sample(interests, k=num_interests)
        user_interests = list(set(user_interests))[:4]  # Max 4 unique interests
        
        users.append({
            'user_id': generic.person.identifier(mask='@@###@'),
            'user_lat': lat,
            'user_lon': lon,
            'user_city': city,
            'user_weather_preference': np.random.choice(['indoor', 'outdoor', 'any'], p=weather_probs),
            'age': age,
            'user_interests': ','.join(user_interests),
            'signup_date': fake.date_time_between(start_date='-2y', end_date='now'),
            'social_connectedness': np.random.poisson(lam=15)
        })
    return pd.DataFrame(users)

def generate_events(n_events=5000):
    events = []
    event_types = [
        'Education & Learning', 'Technology', 'Seasonal & Festivals', 
        'Arts & Culture', 'Entertainment', 'Sports & Fitness', 
        'Business & Networking', 'Health & Wellness', 'Music & Concerts', 
        'Food & Drink', 'Community & Causes', 'Immersive Experiences'
    ]
    current_date = datetime(2025, 3, 27, 11, 48)
    
    for _ in range(n_events):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        event_type = random.choice(event_types)
        start_time = fake.date_time_between(
            start_date=current_date, 
            end_date=current_date + timedelta(days=180)
        )
        
        # Weather and temperature logic
        if event_type in ['Sports & Fitness', 'Seasonal & Festivals']:
            weather_condition = 'Clear' if random.random() < 0.8 else random.choice(['Rain', 'Cloudy'])
        else:
            weather_condition = random.choice(['Clear', 'Cloudy', 'Rain', 'Snow', 'Windy'])
        
        base_temp = {
            'New York': 15, 'London': 12, 'Paris': 16, 'Tokyo': 20, 
            'Sydney': 22, 'Berlin': 14, 'Mumbai': 28, 'São Paulo': 24, 
            'Toronto': 10, 'Dubai': 32
        }[city]
        
        temp_adjustment = {
            'Clear': np.random.uniform(2, 5),
            'Rain': np.random.uniform(-3, 0),
            'Snow': np.random.uniform(-8, -3),
            'Cloudy': np.random.uniform(-1, 2),
            'Windy': np.random.uniform(-2, 1)
        }[weather_condition]
        
        events.append({
            'event_id': generic.person.identifier(mask='@@###@'),
            'title': f"{fake.catch_phrase()} {event_type} in {city}",
            'event_type': event_type,
            'event_lat': lat,
            'event_lon': lon,
            'event_city': city,
            'start_time': start_time,
            'duration': np.random.choice([120, 180, 240, 360, 480]),
            'weather_condition': weather_condition,
            'temperature': round(base_temp + temp_adjustment, 1),
            'attendance_rate': np.random.beta(a=2, b=5) * 100,
            'event_indoor_capability': event_type in ['Education & Learning', 'Technology', 
                                                    'Business & Networking', 'Arts & Culture',
                                                    'Entertainment', 'Immersive Experiences']
        })
    return pd.DataFrame(events)

def generate_interactions(users, events, n_interactions=100000):
    interactions = []
    zipf_param = 1.8  # More skewed distribution
    max_events_per_user = 25
    current_time = datetime(2025, 3, 27, 11, 48)
    
    def safe_sample(df, n):
        return df.sample(n, replace=True) if len(df) > 0 else pd.DataFrame()

    # Phase 1: Core interactions with power-law distribution
    for user in users.itertuples():
        user_interests = set(user.user_interests.split(','))
        base_count = np.random.zipf(zipf_param)
        events_to_attend = min(base_count, max_events_per_user)
        
        # Get interest-matching events with case-insensitive check
        candidate_events = events[
            events['event_type'].apply(
                lambda x: any(i.lower() in x.lower() for i in user_interests)
            )
        ]
        
        if len(candidate_events) == 0:
            continue  # Skip users with no matching events
            
        candidate_sample = safe_sample(candidate_events, 50)
        if len(candidate_sample) == 0:
            continue

        for event in candidate_sample.itertuples():
            if events_to_attend <= 0:
                break
            
            distance = geodesic(
                (user.user_lat, user.user_lon),
                (event.event_lat, event.event_lon)
            ).km
            
            if distance < 50 and random.random() < 0.7:
                interactions.append({
                    'user_id': user.user_id,
                    'event_id': event.event_id,
                    'distance_to_event': distance,
                    'event_type_in_user_interests': 1,
                    'interaction_label': 1
                })
                events_to_attend -= 1

    # Phase 2: Power users with many interactions
    power_users = users.nlargest(500, 'social_connectedness')
    for user in power_users.itertuples():
        additional_events = min(np.random.zipf(zipf_param*1.5), max_events_per_user)
        candidate_events = events.sample(100)
        
        for event in candidate_events.itertuples():
            if additional_events <= 0:
                break
            
            distance = geodesic(
                (user.user_lat, user.user_lon),
                (event.event_lat, event.event_lon)
            ).km
            
            if distance < 100 and random.random() < 0.8:
                interactions.append({
                    'user_id': user.user_id,
                    'event_id': event.event_id,
                    'distance_to_event': distance,
                    'event_type_in_user_interests': int(
                        any(i in event.event_type for i in user.user_interests.split(','))
                    ),
                    'interaction_label': 1
                })
                additional_events -= 1

    # Phase 3: Fill remaining interactions
    while len(interactions) < n_interactions:
        user = users.sample(1).iloc[0]
        event = events.sample(1).iloc[0]
        distance = geodesic(
            (user['user_lat'], user['user_lon']),
            (event['event_lat'], event['event_lon'])
        ).km
        
        interactions.append({
            'user_id': user['user_id'],
            'event_id': event['event_id'],
            'distance_to_event': distance,
            'event_type_in_user_interests': int(
                any(i in event['event_type'] for i in user['user_interests'].split(','))
            ),
            'interaction_label': 1 if random.random() < 0.3 else 0
        })

    return pd.DataFrame(interactions[:n_interactions])

# Generate datasets
users_df = generate_users(10000)
events_df = generate_events(5000)
interactions_df = generate_interactions(users_df, events_df, 150000)

# # Save to CSV
# users_df.to_csv('synthetic_users.csv', index=False)
# events_df.to_csv('synthetic_events.csv', index=False)
# interactions_df.to_csv('synthetic_interactions.csv', index=False)


In [31]:
events_per_user = interactions_df.groupby("user_id").size()
max(events_per_user)

for i in range(1, 50):
    print(f"{events_per_user[events_per_user == i].size} attended {i} events")

0 attended 1 events
0 attended 2 events
0 attended 3 events
0 attended 4 events
0 attended 5 events
0 attended 6 events
0 attended 7 events
0 attended 8 events
0 attended 9 events
618 attended 10 events
653 attended 11 events
672 attended 12 events
690 attended 13 events
663 attended 14 events
671 attended 15 events
628 attended 16 events
692 attended 17 events
619 attended 18 events
710 attended 19 events
626 attended 20 events
685 attended 21 events
696 attended 22 events
688 attended 23 events
688 attended 24 events
0 attended 25 events
0 attended 26 events
0 attended 27 events
0 attended 28 events
0 attended 29 events
0 attended 30 events
0 attended 31 events
0 attended 32 events
0 attended 33 events
0 attended 34 events
0 attended 35 events
0 attended 36 events
0 attended 37 events
0 attended 38 events
0 attended 39 events
0 attended 40 events
0 attended 41 events
0 attended 42 events
0 attended 43 events
0 attended 44 events
0 attended 45 events
0 attended 46 events
0 attended 47

In [29]:
interactions_df.head()

,user_id,event_id,distance_to_event,event_type_in_user_interests,interaction_label
0,ZN105D,BJ353P,16.615926,1,1
1,VS446V,YN703P,15.843688,1,1
2,VS446V,WF293K,10.646002,1,1
3,VS446V,TV389J,3.041735,1,1
4,VS446V,WF293K,10.646002,1,1


In [35]:
len(interactions_df[interactions_df["event_type_in_user_interests"]==1])


12936

In [31]:
interactions_df["interaction_label"].value_counts()

interaction_label
0    95360
1    54640
Name: count, dtype: int64

In [36]:
import numpy as np
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from geopy.distance import geodesic
from scipy.stats import skewnorm, dirichlet
from mimesis import Generic
import hopsworks
import re

# Initialize Faker and set seed for reproducibility
fake = Faker()
generic = Generic('en')

np.random.seed(42)
random.seed(42)

# City coordinates and probabilities
cities = ['New York', 'London', 'Paris', 'Tokyo', 'Sydney', 'Berlin', 'Mumbai', 'São Paulo', 'Toronto', 'Dubai']
city_probs = [0.2, 0.15, 0.1, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.05]
city_coords = {
    'New York': (40.7128, -74.0060), 'London': (51.5074, -0.1278), 'Paris': (48.8566, 2.3522),
    'Tokyo': (35.6762, 139.6503), 'Sydney': (-33.8688, 151.2093), 'Berlin': (52.5200, 13.4050),
    'Mumbai': (19.0760, 72.8777), 'São Paulo': (-23.5505, -46.6333), 'Toronto': (43.6532, -79.3832),
    'Dubai': (25.2048, 55.2708)
}

def generate_location(city):
    base_lat, base_lon = city_coords[city]
    if random.random() < 0.8:
        lat = base_lat + np.random.uniform(-0.1, 0.1)
        lon = base_lon + np.random.uniform(-0.1, 0.1)
    else:
        lat = base_lat + np.random.uniform(-2, 2)
        lon = base_lon + np.random.uniform(-2, 2)
    return lat, lon

def generate_users(n_users=20000):
    users = []
    interests = ['music', 'sports', 'tech', 'food', 'art', 'literature', 'cinema', 'travel', 'fitness', 'fashion']
    
    for _ in range(n_users):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        age = max(18, min(100, int(skewnorm.rvs(5, loc=25, scale=15))))
        weather_probs = dirichlet.rvs([0.3, 0.5, 0.2])[0]
        
        # Ensure at least one interest is selected
        user_interests = random.sample(interests, k=random.randint(1, min(4, len(interests))))
        
        users.append({
            'user_id': generic.person.identifier(mask='@@###@'),
            'user_lat': lat,
            'user_lon': lon,
            'user_city': city,
            'user_weather_preference': np.random.choice(['indoor', 'outdoor', 'any'], p=weather_probs),
            'age': age,
            'user_interests': ','.join(user_interests),  # Join interests into a comma-separated string
            'signup_date': fake.date_time_between(start_date='-2y', end_date='now'),
            'social_connectedness': np.random.poisson(lam=15)
        })
    return pd.DataFrame(users)


def generate_events(n_events=5000):
    events = []
    event_types = [
        'Education & Learning', 'Technology', 'Seasonal & Festivals', 'Arts & Culture', 
        'Entertainment', 'Sports & Fitness', 'Business & Networking', 'Health & Wellness', 
        'Music & Concerts', 'Food & Drink', 'Community & Causes', 'Immersive Experiences'
    ]
    weather_conditions = ['Clear', 'Rain', 'Snow', 'Cloudy', 'Windy']
    weather_probs = [0.5, 0.2, 0.05, 0.2, 0.05]
    
    current_date = datetime(2025, 3, 27, 11, 48)  # Current date and time
    
    for _ in range(n_events):
        event_type = np.random.choice(event_types)
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        
        if event_type in ['Sports & Fitness', 'Seasonal & Festivals']:
            weather_condition = 'Clear' if random.random() < 0.8 else np.random.choice(['Rain', 'Cloudy'])
        elif event_type in ['Education & Learning', 'Technology', 'Business & Networking']:
            weather_condition = np.random.choice(['Clear', 'Cloudy'])
        else:
            weather_condition = np.random.choice(weather_conditions, p=weather_probs)
        
        base_temp = {
            'New York': 15, 'London': 12, 'Paris': 16, 'Tokyo': 20, 
            'Sydney': 22, 'Berlin': 14, 'Mumbai': 28, 'São Paulo': 24, 
            'Toronto': 10, 'Dubai': 32
        }[city]
        
        temp_adjustment = {
            'Clear': np.random.uniform(2, 5),
            'Rain': np.random.uniform(-3, 0),
            'Snow': np.random.uniform(-8, -3),
            'Cloudy': np.random.uniform(-1, 2),
            'Windy': np.random.uniform(-2, 1)
        }[weather_condition]
        
        temperature = round(base_temp + temp_adjustment, 1)
        
        start_time = fake.date_time_between(start_date=current_date, end_date=current_date + timedelta(days=180))
        is_weekend = start_time.weekday() >= 5
        hour_choices = [10, 14, 18] if is_weekend else [9, 13, 18, 19]
        start_time = start_time.replace(hour=np.random.choice(hour_choices))
        
        events.append({
            'event_id': generic.person.identifier(mask='@@###@'),
            'title': f"{fake.catch_phrase()} {event_type} in {city}",
            'event_type': event_type,
            'event_lat': lat,
            'event_lon': lon,
            'event_city': city,
            'start_time': start_time,
            'duration': np.random.choice([120, 180, 240, 360, 480]),
            'weather_condition': weather_condition,
            'temperature': temperature,
            'attendance_rate': np.random.beta(a=2, b=5) * 100,
            'event_indoor_capability': event_type in ['Education & Learning', 'Technology', 'Business & Networking', 
                                               'Arts & Culture', 'Entertainment', 'Immersive Experiences']
        })
    return pd.DataFrame(events)

def calculate_time_weight(interaction_time, current_time, half_life=30):
    time_diff = (current_time - interaction_time).days
    return np.exp(np.log(0.5) * time_diff / half_life)

def generate_interactions(users, events, n_interactions=100000):
    interactions = []
    interaction_types = ['maybe', 'invited & maybe', 'no', 'yes', 'invited & yes', 'invited & no', 'invited']
    attempts = n_interactions * 5
    current_time = datetime(2025, 3, 27, 11, 48)
    
    # Create a user participation bias - some users are much more active than others
    # Using a reversed power law distribution to favor higher event attendance
    participation_factor = 1 - np.random.power(0.5, size=len(users))
    users['participation_factor'] = participation_factor
    
    # Sort users by participation factor (higher values first)
    users = users.sort_values('participation_factor', ascending=False)
    
    # Group events by city to increase the probability of users attending events in their city
    events_by_city = {city: events[events['event_city'] == city] for city in cities}
    
    # Create a dictionary to track user event counts
    user_event_count = {user_id: 0 for user_id in users['user_id']}
    
    for _ in range(attempts):
        if len(interactions) >= n_interactions:
            break
            
        # Sample users with probability proportional to their participation factor
        # This ensures users with higher participation factors get selected more often
        user_probs = users['participation_factor'] / users['participation_factor'].sum()
        user_idx = np.random.choice(range(len(users)), p=user_probs)
        user = users.iloc[user_idx]
        
        # Prioritize events in the user's city with higher probability
        user_city_events = events_by_city.get(user['user_city'], pd.DataFrame())
        if len(user_city_events) > 0 and random.random() < 0.85:
            event = user_city_events.sample(1).iloc[0]
        else:
            event = events.sample(1).iloc[0]
        
        distance = geodesic((user['user_lat'], user['user_lon']), 
                           (event['event_lat'], event['event_lon'])).km
        
        # Adjust distance calculation to favor closer events
        distance_score = np.exp(-distance/50)  # More forgiving distance score (changed from 10 to 50)
        
        weather_score = 1.5 if (event['weather_condition'] == 'Clear' and 
                               user['user_weather_preference'] in ['outdoor', 'any']) else 0.8
        
        # Increase the impact of social connectedness
        social_score = np.log1p(user['social_connectedness']) / 5  # Changed from 10 to 5
        
        # Add participation factor to interaction probability
        participation_boost = user['participation_factor'] * 0.5
        
        # Combine all factors with emphasis on participation
        interaction_prob = 0.5*distance_score + 0.1*weather_score + 0.1*social_score + 0.3*participation_boost
        
        # Increase the maximum distance threshold
        max_distance = 100 if random.random() < 0.8 else 500  # Increased from 50/300 to 100/500
        
        if distance < max_distance and (random.random() < interaction_prob):
            interaction_time = fake.date_time_between(
                start_date=event['start_time'] - timedelta(days=30), 
                end_date=event['start_time']
            )
            time_weight = calculate_time_weight(interaction_time, current_time)
            interaction_prob *= time_weight
            
            # Adjust interaction type probabilities to favor positive responses
            if distance <= 10:
                interaction_type_probs = [0.15, 0.25, 0.05, 0.35, 0.15, 0.03, 0.02]  # More yes and invited & yes
            elif distance <= 40:
                interaction_type_probs = [0.20, 0.20, 0.05, 0.30, 0.15, 0.05, 0.05]
            elif distance <= 100:
                interaction_type_probs = [0.25, 0.15, 0.10, 0.20, 0.15, 0.10, 0.05]
            elif distance <= 200:
                interaction_type_probs = [0.20, 0.10, 0.15, 0.15, 0.15, 0.15, 0.10]
            else:
                interaction_type_probs = [0.15, 0.10, 0.20, 0.15, 0.10, 0.20, 0.10]
            
            # Select interaction type
            interaction_type = np.random.choice(interaction_types, p=interaction_type_probs)
            
            # Determine if this counts as attendance (maybe or yes)
            is_attending = interaction_type in ['maybe', 'invited & maybe', 'yes', 'invited & yes']
            
            # Add to user's event count if attending
            if is_attending:
                user_event_count[user['user_id']] += 1
                
            # Create the interaction record
            interactions.append({
                'interaction_id': generic.person.identifier(mask='@@###@'),
                'user_id': user['user_id'],
                'event_id': event['event_id'],
                'interaction_type': interaction_type,
                'distance_to_event': distance,
                'interaction_label': 1 if is_attending else 0
            })
    
    # Convert to DataFrame
    interactions_df = pd.DataFrame(interactions)
    
    return interactions_df

# Generate data
users_df = generate_users(n_users=20000)
events_df = generate_events(n_events=5000)
interactions_df = generate_interactions(users_df, events_df, n_interactions=100000)

# Calculate and display the distribution of events attended per user
attendance_counts = interactions_df[interactions_df['interaction_label'] == 1].groupby('user_id').size()
attendance_distribution = attendance_counts.value_counts().sort_index()

print("Distribution of events attended per user:")
for count, num_users in attendance_distribution.items():
    print(f"{num_users} attended {count} events")

# # Optional: Save data to CSV files
# users_df.to_csv('users.csv', index=False)
# events_df.to_csv('events.csv', index=False)
# interactions_df.to_csv('interactions.csv', index=False)

Distribution of events attended per user:
2210 attended 1 events
1811 attended 2 events
1736 attended 3 events
1816 attended 4 events
1763 attended 5 events
1753 attended 6 events
1514 attended 7 events
1252 attended 8 events
952 attended 9 events
648 attended 10 events
435 attended 11 events
276 attended 12 events
152 attended 13 events
69 attended 14 events
50 attended 15 events
22 attended 16 events
14 attended 17 events
8 attended 18 events
1 attended 19 events
1 attended 20 events


In [38]:
interactions_df['interaction_label'].value_counts()

interaction_label
1    85861
0    14139
Name: count, dtype: int64

In [40]:
interactions_df.head()

,interaction_id,user_id,event_id,interaction_type,distance_to_event,interaction_label
0,XN454Q,FI641K,TJ649N,invited & maybe,10.159621,1
1,TD269S,ZT737H,ZJ827M,invited & maybe,5.877954,1
2,RU745A,JR639E,JU394E,invited & maybe,5.391734,1
3,UN286X,MS715T,QQ190V,invited & no,10.824373,0
4,OW716Q,TP904C,GE665R,yes,10.946627,1


In [19]:
import numpy as np
import pandas as pd
import random
from faker import Faker
from datetime import datetime, timedelta
from geopy.distance import geodesic
from scipy.stats import skewnorm, dirichlet, zipf
from mimesis import Generic

# Initialize Faker and set seed for reproducibility
fake = Faker()
generic = Generic('en')
np.random.seed(42)
random.seed(42)

# City coordinates and probabilities
cities = ['New York', 'London', 'Paris', 'Tokyo', 'Sydney', 'Berlin', 'Mumbai', 'São Paulo', 'Toronto', 'Dubai']
city_probs = [0.2, 0.15, 0.1, 0.1, 0.05, 0.1, 0.1, 0.05, 0.1, 0.05]
city_coords = {
    'New York': (40.7128, -74.0060), 'London': (51.5074, -0.1278), 'Paris': (48.8566, 2.3522),
    'Tokyo': (35.6762, 139.6503), 'Sydney': (-33.8688, 151.2093), 'Berlin': (52.5200, 13.4050),
    'Mumbai': (19.0760, 72.8777), 'São Paulo': (-23.5505, -46.6333), 'Toronto': (43.6532, -79.3832),
    'Dubai': (25.2048, 55.2708)
}

event_category = [
    'Education',          # Lectures, seminars, or academic events (e.g., university talks)
    'Technology',        # Tech meetups, hackathons, or product launches
    'Arts',              # Art gallery openings, visual arts exhibitions
    'Festival',          # Cultural, music, or seasonal festivals (e.g., summer street fairs)
    'Entertainment',     # General entertainment like comedy shows or magic performances
    'Sports',            # Sporting events (e.g., soccer matches, marathons)
    'Business',          # Networking events, startup pitches, or corporate events
    'Conference',        # Professional or industry conferences (e.g., tech or medical summits)
    'Exhibition',        # Trade shows, museum exhibits, or product showcases
    'Workshop',          # Hands-on learning sessions (e.g., coding or painting workshops)
    'Health',            # Wellness events like yoga classes or mental health talks
    'Music Concerts',    # Live music performances (e.g., rock, jazz, classical)
    'Food Fair',         # Culinary events like food truck festivals or wine tastings
    'Theater',           # Plays, musicals, or performance art
    'Cinema',            # Movie screenings, film festivals, or premieres
    'Dance',             # Dance performances or classes (e.g., ballet, salsa)
    'Literature',        # Book readings, author talks, or poetry slams
    'Charity',           # Fundraising events, charity runs, or volunteer drives
    'Fashion',           # Fashion shows, pop-up boutiques, or designer showcases
    'Gaming',            # Esports tournaments, board game nights, or VR experiences
    'Outdoor Adventure', # Hiking, camping, or adventure sports events
    'Family',            # Kid-friendly events like puppet shows or science fairs
    'Cultural',          # Heritage celebrations, cultural expos, or religious festivals
    'Nightlife',         # Club events, DJ nights, or bar crawls
    'Fitness',           # Group fitness classes, fun runs, or obstacle courses
    'Science',           # Science fairs, planetarium shows, or innovation demos
    'Craft',             # Craft fairs, DIY workshops, or maker markets
    'Photography',       # Photo walks, exhibitions, or photography classes
    'History',           # Historical reenactments, museum tours, or heritage walks
    'Virtual',           # Online events, webinars, or virtual reality meetups
]

def generate_location(city):
    base_lat, base_lon = city_coords[city]
    if random.random() < 0.8:
        lat = base_lat + np.random.uniform(-0.1, 0.1)
        lon = base_lon + np.random.uniform(-0.1, 0.1)
    else:
        lat = base_lat + np.random.uniform(-2, 2)
        lon = base_lon + np.random.uniform(-2, 2)
    return lat, lon

def generate_users(n_users=10000):
    users = []
    interests = [
    'music',              # Interest in music-related events (e.g., concerts, music festivals)
    'sports',            # Interest in sports events (e.g., matches, marathons)
    'technology',              # Interest in technology (e.g., hackathons, tech talks)
    'food',              # Interest in culinary events (e.g., food fairs, wine tastings)
    'art',               # Interest in visual arts (e.g., gallery exhibitions, art workshops)
    'literature',        # Interest in books and writing (e.g., author talks, poetry slams)
    'cinema',            # Interest in movies (e.g., film screenings, festivals)
    'travel',            # Interest in travel-related events (e.g., travel expos, adventure talks)
    'fitness',           # Interest in physical activities (e.g., yoga, fun runs)
    'fashion',           # Interest in fashion events (e.g., fashion shows, pop-up boutiques)
    'festival',          # Interest in festivals (e.g., cultural, seasonal, or music festivals)
    'health',            # Interest in wellness (e.g., mental health talks, wellness workshops)
    'business',          # Interest in professional events (e.g., networking, startup pitches)
    'theater',           # Interest in performing arts (e.g., plays, musicals)
    'dance',             # Interest in dance (e.g., ballet performances, salsa classes)
    'gaming',            # Interest in gaming (e.g., esports, board game nights)
    'outdoor',           # Interest in outdoor activities (e.g., hiking, camping)
    'family',            # Interest in family-friendly events (e.g., puppet shows, science fairs)
    'cultural',          # Interest in cultural events (e.g., heritage celebrations)
    'nightlife',         # Interest in nightlife (e.g., club events, DJ nights)
    'science',           # Interest in science (e.g., planetarium shows, science fairs)
    'craft',             # Interest in crafts (e.g., DIY workshops, craft fairs)
    'photography',       # Interest in photography (e.g., photo walks, exhibitions)
    'gardening',         # Interest in gardening (e.g., plant swaps, garden tours)
    'history',           # Interest in historical events (e.g., reenactments, heritage walks)
    'education',         # Interest in learning (e.g., lectures, educational workshops)
    'charity',           # Interest in charitable causes (e.g., fundraisers, volunteer events)
]
    genders = ['male', 'female', 'other']
    gender_probs = [0.48, 0.48, 0.04]
    
    for _ in range(n_users):
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        age = max(18, min(100, int(skewnorm.rvs(5, loc=25, scale=15))))
        weather_probs = dirichlet.rvs([0.3, 0.5, 0.2])[0]
        gender = np.random.choice(genders, p=gender_probs)
        # Power-law: more users with more interests
        num_interests = min(max(zipf.rvs(2.0), 1), 5)
        user_interests = random.sample(interests, k=num_interests)
        users.append({
            'user_id': generic.person.identifier(mask='@@###@'),
            'gender': gender,
            'user_lat': lat,
            'user_lon': lon,
            'user_city': city,
            'indoor_outdoor_preference': np.random.choice(['indoor', 'outdoor', 'any'], p=weather_probs),
            'age': age,
            'user_interests': ','.join(user_interests),
            'signup_date': fake.date_time_between(start_date='-2y', end_date='now')
        })
    return pd.DataFrame(users)

def generate_events(n_events=3000):
    events = []
    weather_conditions = ['Clear', 'Rain', 'Snow', 'Cloudy', 'Windy']
    weather_probs = [0.5, 0.2, 0.05, 0.2, 0.05]
    current_date = datetime(2025, 3, 27, 11, 48)
    for _ in range(n_events):
        cat = np.random.choice(event_category)
        city = np.random.choice(cities, p=city_probs)
        lat, lon = generate_location(city)
        if cat in ['Sports', 'Festival']:
            weather_condition = 'Clear' if random.random() < 0.8 else np.random.choice(['Rain', 'Cloudy'])
        elif cat in ['Education', 'Technology', 'Business', 'Conference', 'Exhibition', 'Workshop']:
            weather_condition = np.random.choice(['Clear', 'Cloudy'])
        else:
            weather_condition = np.random.choice(weather_conditions, p=weather_probs)
        base_temp = {
            'New York': 15, 'London': 12, 'Paris': 16, 'Tokyo': 20, 
            'Sydney': 22, 'Berlin': 14, 'Mumbai': 28, 'São Paulo': 24, 
            'Toronto': 10, 'Dubai': 32
        }[city]
        temp_adjustment = {
            'Clear': np.random.uniform(2, 5),
            'Rain': np.random.uniform(-3, 0),
            'Snow': np.random.uniform(-8, -3),
            'Cloudy': np.random.uniform(-1, 2),
            'Windy': np.random.uniform(-2, 1)
        }[weather_condition]
        temperature = round(base_temp + temp_adjustment, 1)
        start_time = fake.date_time_between(start_date=current_date, end_date=current_date + timedelta(days=180))
        is_weekend = start_time.weekday() >= 5
        hour_choices = [10, 14, 18] if is_weekend else [9, 13, 18, 19]
        start_time = start_time.replace(hour=np.random.choice(hour_choices))
        events.append({
            'event_id': generic.person.identifier(mask='@@###@'),
            'title': f"{fake.catch_phrase()} {cat} in {city}",
            'event_category': cat,
            'event_lat': lat,
            'event_lon': lon,
            'event_city': city,
            'start_time': start_time,
            'duration': np.random.choice([120, 180, 240, 360, 480]),
            'weather_condition': weather_condition,
            'temperature': temperature,
            'attendance_rate': np.random.beta(a=2, b=5) * 100,
            'event_indoor_capability': cat in [
                'Education', 'Technology', 'Business', 'Conference', 'Exhibition', 'Workshop', 
                'Arts', 'Entertainment', 'Music Concerts', 'Theater', 'Cinema'
            ]
        })
    return pd.DataFrame(events)

def calculate_time_weight(interaction_time, current_time, half_life=30):
    time_diff = (current_time - interaction_time).days
    return np.exp(np.log(0.5) * time_diff / half_life)

def generate_interactions(users, events, n_interactions=120000):
    interactions = []
    interaction_types_positive = ['yes', 'invited & yes']
    interaction_types_other = ['maybe', 'invited & maybe', 'no', 'invited & no', 'invited']
    current_time = datetime(2025, 3, 27, 11, 48)
    min_positive_interactions = 10
    
    # Assign total interactions per user (10–24, ensuring at least 10 positive)
    user_interaction_counts = {user_id: np.random.randint(10, 25) for user_id in users['user_id']}
    user_positive_counts = {user_id: 0 for user_id in users['user_id']}
    
    # Generate interactions for each user
    for user_id, total_interactions in user_interaction_counts.items():
        user = users[users['user_id'] == user_id].iloc[0]
        user_interests = set(user['user_interests'].split(','))
        
        # Sample events, prioritizing interest matches
        interest_events = events[events['event_category'].apply(lambda x: any(i.lower() in x.lower() for i in user_interests))]
        sample_size = max(total_interactions, len(interest_events))
        chosen_events = interest_events.sample(n=sample_size, replace=True) if len(interest_events) > 0 else events.sample(n=sample_size, replace=True)
        
        for _, event in chosen_events.head(total_interactions).iterrows():
            distance = geodesic((user['user_lat'], user['user_lon']), (event['event_lat'], event['event_lon'])).km
            interaction_time = fake.date_time_between(start_date=event['start_time'] - timedelta(days=30), end_date=event['start_time'])
            interest_match = int(any(i.lower() in event['event_category'].lower() for i in user_interests))
            
            # Weather penalty for outdoor events in bad weather
            weather_penalty = 0.5 if (not event['event_indoor_capability'] and event['weather_condition'] in ['Rain', 'Snow'] and user['indoor_outdoor_preference'] == 'indoor') else 0.2 if (not event['event_indoor_capability'] and event['weather_condition'] in ['Rain', 'Snow'] and user['indoor_outdoor_preference'] == 'any') else 0
            
            # Prioritize positive interactions until minimum is met
            needs_positive = user_positive_counts[user_id] < min_positive_interactions
            is_positive = needs_positive or (random.random() > 0.5 and random.random() > weather_penalty)
            interaction_type = random.choice(interaction_types_positive) if is_positive else random.choice(interaction_types_other)
            interaction_label = 1 if is_positive else 0
            
            interactions.append({
                'interaction_id': generic.person.identifier(mask='@@###@'),
                'user_id': user_id,
                'event_id': event['event_id'],
                'interaction_type': interaction_type,
                'distance_to_event': distance,
                'interaction_label': interaction_label,
                'interest_match': interest_match
            })
            
            if is_positive:
                user_positive_counts[user_id] += 1
    
    # Fill remaining interactions to reach n_interactions
    while len(interactions) < n_interactions:
        user_id = random.choice(users['user_id'].tolist())
        user = users[users['user_id'] == user_id].iloc[0]
        user_interests = set(user['user_interests'].split(','))
        event = events.sample(n=1).iloc[0]
        
        distance = geodesic((user['user_lat'], user['user_lon']), (event['event_lat'], event['event_lon'])).km
        interaction_time = fake.date_time_between(start_date=event['start_time'] - timedelta(days=30), end_date=event['start_time'])
        interest_match = int(any(i.lower() in event['event_category'].lower() for i in user_interests))
        
        weather_penalty = 0.5 if (not event['event_indoor_capability'] and event['weather_condition'] in ['Rain', 'Snow'] and user['indoor_outdoor_preference'] == 'indoor') else 0.2 if (not event['event_indoor_capability'] and event['weather_condition'] in ['Rain', 'Snow'] and user['indoor_outdoor_preference'] == 'any') else 0
        is_positive = random.random() > 0.5 and random.random() > weather_penalty
        interaction_type = random.choice(interaction_types_positive) if is_positive else random.choice(interaction_types_other)
        interaction_label = 1 if is_positive else 0
        
        interactions.append({
            'interaction_id': generic.person.identifier(mask='@@###@'),
            'user_id': user_id,
            'event_id': event['event_id'],
            'interaction_type': interaction_type,
            'distance_to_event': distance,
            'interaction_label': interaction_label,
            'interest_match': interest_match
        })
        if is_positive:
            user_positive_counts[user_id] += 1
    
    return pd.DataFrame(interactions)


# Generate datasets
users_df = generate_users(10000)
events_df = generate_events(10000)
interactions_df = generate_interactions(users_df, events_df, n_interactions=100000)

# # Save to CSV
# users_df.to_csv('synthetic_users.csv', index=False)
# events_df.to_csv('synthetic_events.csv', index=False)
# interactions_df.to_csv('synthetic_interactions.csv', index=False)


In [30]:
# Calculate and display the distribution of events attended per user
attendance_counts = interactions_df.groupby('user_id').size()
attendance_distribution = attendance_counts.value_counts().sort_index()

print("Distribution of events attended per user:")
for count, num_users in attendance_distribution.items():
    print(f"{num_users} attended {count} events")

# Calculate the number of interactions for users who attended 10 or more events
users_with_10_or_more_events = attendance_distribution[attendance_distribution.index >= 11].sum()
print(f"Number of interactions for users who attended 10 or more events: {users_with_10_or_more_events}")


Distribution of events attended per user:
618 attended 10 events
653 attended 11 events
672 attended 12 events
690 attended 13 events
663 attended 14 events
671 attended 15 events
628 attended 16 events
692 attended 17 events
619 attended 18 events
710 attended 19 events
626 attended 20 events
685 attended 21 events
696 attended 22 events
688 attended 23 events
688 attended 24 events
Number of interactions for users who attended 10 or more events: 9381


In [41]:
# # Save to CSV
users_df.to_csv('synthetic_users.csv', index=False)
events_df.to_csv('synthetic_events.csv', index=False)
interactions_df.to_csv('synthetic_interactions.csv', index=False)

In [22]:
len(interactions_df)

170741

In [21]:
interactions_df['interaction_label'].value_counts()

interaction_label
1    134067
0     36674
Name: count, dtype: int64

In [46]:
interactions_df.head()

,interaction_id,user_id,event_id,interaction_type,distance_to_event,interaction_label,interest_match
0,DY865T,LH639X,QH593N,invited & yes,19.390390,1,1
1,JK377A,LH639X,PJ960I,invited,9718.955250,1,1
2,IU133S,LH639X,CC961P,no,7254.504840,1,1
3,HU050G,LH639X,IU130U,no,9249.921518,0,1
4,BL008W,LH639X,UY375V,invited & no,325.393142,1,1


In [52]:
print(len(interactions_df[interactions_df['interest_match'] == 1]))
len(interactions_df[interactions_df['interest_match'] == 0])


106599


23652

In [53]:
len(interactions_df)

130251

In [33]:
merged_df = users_df.merge(interactions_df, on="user_id",how="inner")\
    .merge(events_df,on="event_id",how="inner")

print(len(merged_df))
merged_df.head()

170764


,user_id,gender,user_lat,user_lon,user_city,indoor_outdoor_preference,age,user_interests,signup_date,interaction_id,...,event_category,event_lat,event_lon,event_city,start_time,duration,weather_condition,temperature,attendance_rate,event_indoor_capability
0,QY831O,male,48.946743,2.398599,Paris,outdoor,40,music,2024-04-24 10:30:13.682443,RX851G,...,Music Concerts,52.610034,13.3988,Berlin,2025-05-27 18:04:34.907373,180,Clear,16.2,33.344309,True
1,ZX781T,male,35.649461,139.679388,Tokyo,outdoor,29,music,2024-04-14 20:27:55.801585,IJ828R,...,Music Concerts,52.610034,13.3988,Berlin,2025-05-27 18:04:34.907373,180,Clear,16.2,33.344309,True
2,VM571H,male,51.552374,-0.105399,London,indoor,42,"music,festival,science,food,craft",2023-11-12 23:22:33.194089,AZ647H,...,Music Concerts,52.610034,13.3988,Berlin,2025-05-27 18:04:34.907373,180,Clear,16.2,33.344309,True
3,WZ425F,other,19.150239,72.811710,Mumbai,indoor,23,"music,festival",2024-03-06 17:28:51.046101,JR744U,...,Music Concerts,52.610034,13.3988,Berlin,2025-05-27 18:04:34.907373,180,Clear,16.2,33.344309,True
4,XA863B,male,41.541536,-72.126511,New York,indoor,29,music,2024-01-07 19:41:16.405027,ON454C,...,Music Concerts,52.610034,13.3988,Berlin,2025-05-27 18:04:34.907373,180,Clear,16.2,33.344309,True


In [39]:
len(merged_df[merged_df["interaction_label"]==0])

36680

In [40]:
merged_df.to_csv("test_merged_data.csv")

In [59]:
import tensorflow as tf
from sentence_transformers import SentenceTransformer
from tensorflow.keras.layers import TextVectorization, Normalization

# 1. Initialize Sentence Transformers
user_text_encoder = SentenceTransformer('all-MiniLM-L6-v2')
event_text_encoder = SentenceTransformer('all-MiniLM-L6-v2')

# 2. Feature Processing Layers
geo_normalizer = Normalization(axis=-1)
temp_normalizer = Normalization(axis=-1)
age_normalizer = Normalization(axis=-1)

# 3. User Tower
class UserTower(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.text_encoder = tf.keras.layers.Lambda(
            lambda x: user_text_encoder.encode(x.numpy().astype(str), convert_to_tensor=True)
        )
        self.geo_norm = geo_normalizer
        self.age_norm = age_normalizer
        self.dense = tf.keras.layers.Dense(256)

    def call(self, inputs):
        # Process interests
        interest_emb = self.text_encoder(inputs["user_interests"])
        
        # Process demographics
        geo = self.geo_norm(tf.stack([inputs["user_lat"], inputs["user_lon"]], axis=1))
        age = self.age_norm(tf.reshape(inputs["age"], (-1, 1)))
        
        # Combine features
        combined = tf.concat([
            tf.cast(interest_emb, tf.float32),
            geo,
            age
        ], axis=1)
        
        return self.dense(combined)

# 4. Event Tower  
class EventTower(tf.keras.Model):
    def __init__(self):
        super().__init__()
        self.text_encoder = tf.keras.layers.Lambda(
            lambda x: event_text_encoder.encode(x.numpy().astype(str), convert_to_tensor=True)
        )
        self.geo_norm = geo_normalizer
        self.temp_norm = temp_normalizer
        self.dense = tf.keras.layers.Dense(256)

    def call(self, inputs):
        # Process event context
        context = inputs["event_title"] + " " + inputs["event_category"]
        context_emb = self.text_encoder(context)
        
        # Process environmental factors
        geo = self.geo_norm(tf.stack([inputs["event_lat"], inputs["event_lon"]], axis=1))
        temp = self.temp_norm(tf.reshape(inputs["temperature"], (-1, 1)))
        
        # Combine features
        combined = tf.concat([
            tf.cast(context_emb, tf.float32),
            geo,
            temp
        ], axis=1)
        
        return self.dense(combined)

# 5. Two-Tower Model
class TwoTowerModel(tfrs.models.Model):
    def __init__(self):
        super().__init__()
        self.user_tower = UserTower()
        self.event_tower = EventTower()
        self.task = tfrs.tasks.Retrieval(
            metrics=tfrs.metrics.FactorizedTopK(
                candidates=events.batch(128).map(self.event_tower)
            )
        )

    def train_step(self, features):
        with tf.GradientTape() as tape:
            user_embeddings = self.user_tower({
                "user_interests": features["user_interests"],
                "user_lat": features["user_lat"],
                "user_lon": features["user_lon"],
                "age": features["age"]
            })
            
            event_embeddings = self.event_tower({
                "event_title": features["event_title"],
                "event_category": features["event_category"],
                "event_lat": features["event_lat"],
                "event_lon": features["event_lon"],
                "temperature": features["temperature"]
            })

            loss = self.task(user_embeddings, event_embeddings)

        gradients = tape.gradient(loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))

        return {"loss": loss}

# 6. Data Preparation
def prepare_dataset(interactions_df):
    return tf.data.Dataset.from_tensor_slices({
        "user_interests": interactions_df['user_interests'],
        "user_lat": interactions_df['user_lat'],
        "user_lon": interactions_df['user_lon'],
        "age": interactions_df['age'],
        "event_title": interactions_df['event_title'],
        "event_category": interactions_df['event_category'],
        "event_lat": interactions_df['event_lat'],
        "event_lon": interactions_df['event_lon'],
        "temperature": interactions_df['temperature']
    }).batch(512).cache()

# 7. Training
def train_model(users, events, interactions):
    # Prepare data
    train_ds = prepare_dataset(interactions)
    event_ds = prepare_dataset(events)
    
    # Initialize towers
    model = TwoTowerModel()
    model.compile(optimizer=tf.keras.optimizers.Adam(0.001))
    
    # Adapt normalizers
    geo_data = tf.concat([users[['user_lat', 'user_lon']], events[['event_lat', 'event_lon']]], axis=0)
    model.user_tower.geo_norm.adapt(geo_data)
    model.event_tower.geo_norm.adapt(geo_data)
    
    model.user_tower.age_norm.adapt(users['age'])
    model.event_tower.temp_norm.adapt(events['temperature'])
    
    # Fine-tune
    model.fit(train_ds, epochs=10)
    
    return model

# 8. Usage
model = train_model(users_df, events_df, interactions_df)


2025-05-02 00:49:09,330 INFO: Use pytorch device_name: cpu
2025-05-02 00:49:09,344 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

2025-05-02 00:49:15,034 INFO: Use pytorch device_name: cpu
2025-05-02 00:49:15,036 INFO: Load pretrained SentenceTransformer: all-MiniLM-L6-v2


NameError: name 'tfrs' is not defined

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers
import tensorflow_recommenders as tfrs
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
import datetime

# Load the generated datasets
users_df = pd.read_csv('synthetic_users.csv')
events_df = pd.read_csv('synthetic_events.csv')
interactions_df = pd.read_csv('synthetic_interactions.csv')

print(f"Loaded {len(users_df)} users, {len(events_df)} events, and {len(interactions_df)} interactions")

# Convert date columns to datetime
users_df['signup_date'] = pd.to_datetime(users_df['signup_date'])
events_df['start_time'] = pd.to_datetime(events_df['start_time'])
interactions_df['interaction_time'] = pd.to_datetime(interactions_df['interaction_time'])

# Data overview
print("\nUsers data sample:")
print(users_df.head())
print("\nEvents data sample:")
print(events_df.head())
print("\nInteractions data sample:")
print(interactions_df.head())

# Let's analyze the data to understand it better
def analyze_data():
    print("\n---- Data Analysis ----")
    
    # User weather preferences distribution
    print("\nUser weather preferences:")
    print(users_df['user_weather_preference'].value_counts(normalize=True))
    
    # Event categories distribution
    print("\nEvent categories:")
    print(events_df['event_category'].value_counts())
    
    # Weather conditions distribution
    print("\nWeather conditions:")
    print(events_df['weather_condition'].value_counts())
    
    # Interaction types distribution
    print("\nInteraction types:")
    print(interactions_df['interaction_type'].value_counts())
    
    # Interaction label distribution
    print("\nInteraction labels:")
    print(interactions_df['interaction_label'].value_counts(normalize=True))
    
    # Plot the distribution of ages
    plt.figure(figsize=(10, 6))
    sns.histplot(users_df['age'], bins=20)
    plt.title('Distribution of User Ages')
    plt.savefig('user_age_distribution.png')
    
    # Plot the distribution of temperatures at events
    plt.figure(figsize=(10, 6))
    sns.histplot(events_df['temperature'], bins=20)
    plt.title('Distribution of Event Temperatures')
    plt.savefig('event_temperature_distribution.png')
    
    # Plot interaction distances
    plt.figure(figsize=(10, 6))
    sns.histplot(interactions_df['interaction_distance_to_event'].clip(0, 500), bins=50)
    plt.title('Distribution of Interaction Distances (km, capped at 500km)')
    plt.savefig('interaction_distance_distribution.png')

analyze_data()

# Feature Engineering for Weather-Based Recommendation

def engineer_features():
    print("\n---- Feature Engineering ----")
    
    # Merge interactions with user and event data
    merged_df = interactions_df.merge(users_df, on='user_id', how='left')
    merged_df = merged_df.merge(events_df, on='event_id', how='left')
    
    # Create weather preference match feature
    def weather_preference_match(row):
        if row['user_weather_preference'] == 'any':
            return 1.0
        elif row['user_weather_preference'] == 'indoor' and row['event_indoor_capability']:
            return 1.0
        elif row['user_weather_preference'] == 'outdoor' and not row['event_indoor_capability']:
            return 1.0
        else:
            return 0.5
    
    merged_df['weather_preference_match'] = merged_df.apply(weather_preference_match, axis=1)
    
    # Create temperature comfort feature (closer to preferred temperature = higher score)
    # Assume comfort zone is between 18-24°C
    def temp_comfort_score(temp):
        if 18 <= temp <= 24:
            return 1.0
        elif 15 <= temp < 18 or 24 < temp <= 27:
            return 0.8
        elif 10 <= temp < 15 or 27 < temp <= 30:
            return 0.6
        elif 5 <= temp < 10 or 30 < temp <= 35:
            return 0.4
        else:
            return 0.2
    
    merged_df['temperature_comfort'] = merged_df['temperature'].apply(temp_comfort_score)
    
    # Create weather condition score
    weather_scores = {
        'Clear': 1.0,
        'Cloudy': 0.8,
        'Windy': 0.6,
        'Rain': 0.4,
        'Snow': 0.2
    }
    merged_df['weather_condition_score'] = merged_df['weather_condition'].map(weather_scores)
    
    # Create recency feature
    current_time = datetime.datetime(2025, 3, 27, 11, 48)
    merged_df['days_since_interaction'] = (current_time - merged_df['interaction_time']).dt.total_seconds() / (24 * 3600)
    merged_df['interaction_recency'] = np.exp(-0.05 * merged_df['days_since_interaction'])  # Exponential decay
    
    # Create time-to-event feature
    merged_df['days_to_event'] = (merged_df['start_time'] - merged_df['interaction_time']).dt.total_seconds() / (24 * 3600)
    merged_df['days_to_event_scaled'] = np.clip(merged_df['days_to_event'] / 30, 0, 1)  # Normalize to [0,1]
    
    # Create weekend flag
    merged_df['is_weekend'] = merged_df['start_time'].dt.dayofweek >= 5
    
    # Create distance feature (exponential decay based on distance)
    merged_df['distance_score'] = np.exp(-0.01 * merged_df['interaction_distance_to_event'])
    
    # One-hot encode categorical variables
    categorical_cols = ['gender', 'user_city', 'user_weather_preference', 
                        'event_category', 'event_city', 'weather_condition',
                        'interaction_type']
    
    for col in categorical_cols:
        dummies = pd.get_dummies(merged_df[col], prefix=col)
        merged_df = pd.concat([merged_df, dummies], axis=1)
    
    # Extract user interests as features
    all_interests = set()
    for interests in users_df['user_interests'].str.split(','):
        all_interests.update(interests)
    
    for interest in all_interests:
        merged_df[f'interest_{interest}'] = merged_df['user_interests'].str.contains(interest).astype(int)
    
    # Scale numerical features
    scaler = StandardScaler()
    numerical_cols = ['age', 'social_connectedness', 'temperature', 'attendance_rate',
                      'interaction_distance_to_event', 'days_to_event']
    
    merged_df[numerical_cols] = scaler.fit_transform(merged_df[numerical_cols])
    
    return merged_df

merged_df = engineer_features()
print("\nFeature-engineered data sample:")
print(merged_df.head())

# Split data into train and test sets
train_df, test_df = train_test_split(merged_df, test_size=0.2, random_state=42)

# PART 1: RETRIEVAL MODEL USING TENSORFLOW RECOMMENDERS

def build_retrieval_model():
    print("\n---- Building Retrieval Model ----")
    
    # Prepare data for TF-Recommenders
    # Convert to TensorFlow datasets
    user_ids = tf.constant(train_df['user_id'].unique())
    event_ids = tf.constant(events_df['event_id'].unique())
    
    # Create mappings
    user_to_features = {}
    for _, user in tqdm(users_df.iterrows(), total=len(users_df), desc="Processing users"):
        user_id = user['user_id']
        user_to_features[user_id] = {
            'gender': user['gender'],
            'city': user['user_city'],
            'weather_preference': user['user_weather_preference'],
            'age': float(user['age']),
            'social_connectedness': float(user['social_connectedness']),
            'interests': user['user_interests']
        }
    
    event_to_features = {}
    for _, event in tqdm(events_df.iterrows(), total=len(events_df), desc="Processing events"):
        event_id = event['event_id']
        event_to_features[event_id] = {
            'category': event['event_category'],
            'city': event['event_city'],
            'weather': event['weather_condition'],
            'temperature': float(event['temperature']),
            'indoor_capability': bool(event['event_indoor_capability']),
            'is_weekend': bool(event['start_time'].dayofweek >= 5)
        }
    
    # Create training data
    train_interactions = tf.data.Dataset.from_tensor_slices({
        'user_id': tf.constant(train_df['user_id'].values),
        'event_id': tf.constant(train_df['event_id'].values),
        'label': tf.constant(train_df['interaction_label'].values, dtype=tf.float32)
    })
    
    # Create test data
    test_interactions = tf.data.Dataset.from_tensor_slices({
        'user_id': tf.constant(test_df['user_id'].values),
        'event_id': tf.constant(test_df['event_id'].values),
        'label': tf.constant(test_df['interaction_label'].values, dtype=tf.float32)
    })
    
    # Define vocabulary sizes
    user_vocab_size = len(user_ids)
    event_vocab_size = len(event_ids)
    embedding_dim = 128
    
    # Define model architecture
    class WeatherEventRetrievalModel(tfrs.Model):
        def __init__(self):
            super().__init__()
            
            # User embeddings
            self.user_embedding_layer = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=user_ids, mask_token=None),
                tf.keras.layers.Embedding(user_vocab_size + 1, embedding_dim)
            ])
            
            # Event embeddings
            self.event_embedding_layer = tf.keras.Sequential([
                tf.keras.layers.StringLookup(vocabulary=event_ids, mask_token=None),
                tf.keras.layers.Embedding(event_vocab_size + 1, embedding_dim)
            ])
            
            # Task: retrieval task that optimizes for user-event matches
            self.task = tfrs.tasks.Retrieval(
                metrics=tfrs.metrics.FactorizedTopK(
                    candidates=tf.data.Dataset.from_tensor_slices(event_ids).map(
                        lambda event_id: (event_id, self.event_embedding_layer(event_id))
                    )
                )
            )
        
        def compute_loss(self, features, training=False):
            user_embeddings = self.user_embedding_layer(features["user_id"])
            event_embeddings = self.event_embedding_layer(features["event_id"])
            
            return self.task(user_embeddings, event_embeddings, compute_metrics=not training)
        
        def call(self, features):
            return {
                "user_embeddings": self.user_embedding_layer(features["user_id"]),
                "event_embeddings": self.event_embedding_layer(features["event_id"])
            }
    
    # Build and compile model
    retrieval_model = WeatherEventRetrievalModel()
    retrieval_model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))
    
    # Shuffle and batch the data
    cached_train = train_interactions.shuffle(10000).batch(8192).cache()
    cached_test = test_interactions.batch(4096).cache()
    
    # Train the model
    history = retrieval_model.fit(cached_train, epochs=5, validation_data=cached_test)
    
    # Create the retrieval index for serving
    index = tfrs.layers.factorized_top_k.BruteForce(retrieval_model.user_embedding_layer)
    index.index_from_dataset(
        tf.data.Dataset.from_tensor_slices((event_ids)).map(lambda event_id: (
            event_id, 
            retrieval_model.event_embedding_layer(event_id)
        ))
    )
    
    # Save the model
    tf.saved_model.save(index, "weather_event_retrieval_model")
    
    return index, retrieval_model, history

retrieval_index, retrieval_model, retrieval_history = build_retrieval_model()

# PART 2: RANKING MODEL USING CATBOOST

def build_ranking_model():
    print("\n---- Building Ranking Model ----")
    
    # Prepare features for CatBoost
    # Select features for ranking
    feature_cols = [
        # User features
        'age', 'social_connectedness',
        # Event features
        'temperature', 'attendance_rate', 'event_indoor_capability',
        # Interaction features
        'interaction_distance_to_event', 'days_to_event',
        # Engineered features
        'weather_preference_match', 'temperature_comfort', 'weather_condition_score',
        'interaction_recency', 'distance_score', 'is_weekend',
        'interest_match'
    ]
    
    # Add one-hot encoded columns
    for col in merged_df.columns:
        if col.startswith(('gender_', 'user_city_', 'user_weather_preference_', 
                           'event_category_', 'event_city_', 'weather_condition_',
                           'interaction_type_', 'interest_')):
            feature_cols.append(col)
    
    # Define categorical features for CatBoost
    categorical_feature_indices = []
    for i, col in enumerate(feature_cols):
        if col.startswith(('gender_', 'user_city_', 'user_weather_preference_', 
                          'event_category_', 'event_city_', 'weather_condition_',
                          'interaction_type_', 'interest_', 'is_weekend', 
                          'event_indoor_capability')):
            categorical_feature_indices.append(i)
    
    # Prepare train and test sets
    X_train = train_df[feature_cols]
    y_train = train_df['interaction_label']
    X_test = test_df[feature_cols]
    y_test = test_df['interaction_label']
    
    print(f"Training set shape: {X_train.shape}")
    print(f"Test set shape: {X_test.shape}")
    
    # Configure and train CatBoost model
    catboost_params = {
        'iterations': 500,
        'learning_rate': 0.03,
        'depth': 6,
        'loss_function': 'Logloss',
        'eval_metric': 'AUC',
        'random_seed': 42,
        'verbose': 100,
        'cat_features': categorical_feature_indices
    }
    
    ranker = CatBoostClassifier(**catboost_params)
    ranker.fit(X_train, y_train, eval_set=(X_test, y_test), plot=True)
    
    # Evaluate the ranking model
    y_pred_proba = ranker.predict_proba(X_test)[:, 1]
    y_pred = ranker.predict(X_test)
    
    print("\nRanking Model Evaluation:")
    print(f"ROC AUC Score: {roc_auc_score(y_test, y_pred_proba):.4f}")
    print(f"Accuracy Score: {accuracy_score(y_test, y_pred):.4f}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'Feature': feature_cols,
        'Importance': ranker.feature_importances_
    }).sort_values(by='Importance', ascending=False)
    
    plt.figure(figsize=(12, 8))
    sns.barplot(x='Importance', y='Feature', data=feature_importance.head(20))
    plt.title('Top 20 Features by Importance')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    
    # Save the model
    ranker.save_model('weather_event_ranking_model.cbm')
    
    return ranker, feature_importance

ranker, feature_importance = build_ranking_model()

# PART 3: FULL RECOMMENDATION PIPELINE

def create_recommendation_pipeline():
    print("\n---- Creating Recommendation Pipeline ----")
    
    def recommend_events_for_user(user_id, top_k=20, final_k=10):
        """
        End-to-end recommendation function using both retrieval and ranking models
        
        Args:
            user_id: The ID of the user to recommend events for
            top_k: Number of candidates to retrieve from retrieval model
            final_k: Final number of recommendations after ranking
            
        Returns:
            DataFrame with ranked event recommendations
        """
        # Step 1: Get user data
        user = users_df[users_df['user_id'] == user_id].iloc[0]
        
        # Step 2: Retrieve candidate events using the retrieval model
        _, candidates = retrieval_index(tf.constant([user_id]), k=top_k)
        candidate_event_ids = [event_id.numpy().decode('utf-8') for event_id in candidates[0]]
        
        # Get candidate events data
        candidate_events = events_df[events_df['event_id'].isin(candidate_event_ids)]
        
        # Step 3: Prepare features for each user-event pair for ranking
        ranking_features = []
        
        for _, event in candidate_events.iterrows():
            # Create a synthetic interaction
            synthetic_interaction = {
                'user_id': user_id,
                'event_id': event['event_id'],
                'interaction_time': datetime.datetime(2025, 3, 27, 11, 48),  # Current time
                'interaction_distance_to_event': geodesic(
                    (user['user_lat'], user['user_lon']),
                    (event['event_lat'], event['event_lon'])
                ).km
            }
            
            # Create a DataFrame with this single interaction
            interaction_df = pd.DataFrame([synthetic_interaction])
            
            # Merge with user and event data
            merged = pd.DataFrame([synthetic_interaction]).merge(
                pd.DataFrame([user]), on='user_id', how='left'
            ).merge(
                pd.DataFrame([event]), on='event_id', how='left'
            )
            
            # Apply the same feature engineering as in training
            # Weather preference match
            if user['user_weather_preference'] == 'any':
                merged['weather_preference_match'] = 1.0
            elif user['user_weather_preference'] == 'indoor' and event['event_indoor_capability']:
                merged['weather_preference_match'] = 1.0
            elif user['user_weather_preference'] == 'outdoor' and not event['event_indoor_capability']:
                merged['weather_preference_match'] = 1.0
            else:
                merged['weather_preference_match'] = 0.5
            
            # Temperature comfort
            def temp_comfort_score(temp):
                if 18 <= temp <= 24:
                    return 1.0
                elif 15 <= temp < 18 or 24 < temp <= 27:
                    return 0.8
                elif 10 <= temp < 15 or 27 < temp <= 30:
                    return 0.6
                elif 5 <= temp < 10 or 30 < temp <= 35:
                    return 0.4
                else:
                    return 0.2
            
            merged['temperature_comfort'] = temp_comfort_score(event['temperature'])
            
            # Weather condition score
            weather_scores = {
                'Clear': 1.0,
                'Cloudy': 0.8,
                'Windy': 0.6,
                'Rain': 0.4,
                'Snow': 0.2
            }
            merged['weather_condition_score'] = weather_scores.get(event['weather_condition'], 0.5)
            
            # Distance score
            merged['distance_score'] = np.exp(-0.01 * merged['interaction_distance_to_event'])
            
            # Time features
            current_time = datetime.datetime(2025, 3, 27, 11, 48)
            merged['days_to_event'] = (merged['start_time'] - current_time).dt.total_seconds() / (24 * 3600)
            merged['interaction_recency'] = 1.0  # This is a new interaction
            merged['is_weekend'] = merged['start_time'].dt.dayofweek >= 5
            
            # Interest match
            user_interests = set(user['user_interests'].split(','))
            merged['interest_match'] = int(any(i.lower() in event['event_category'].lower() for i in user_interests))
            
            # One-hot encoding of categorical variables
            for col, col_value in [
                ('gender', user['gender']),
                ('user_city', user['user_city']),
                ('user_weather_preference', user['user_weather_preference']),
                ('event_category', event['event_category']),
                ('event_city', event['event_city']),
                ('weather_condition', event['weather_condition']),
                ('interaction_type', 'maybe')  # Default value for new interactions
            ]:
                for unique_val in merged_df[col].unique():
                    merged[f'{col}_{unique_val}'] = 1 if col_value == unique_val else 0
            
            # Interest features
            for interest in all_interests:
                merged[f'interest_{interest}'] = 1 if interest in user_interests else 0
            
            ranking_features.append(merged)
        
        if not ranking_features:
            return pd.DataFrame()
            
        ranking_df = pd.concat(ranking_features, ignore_index=True)
        
        # Use the same feature columns as in training
        feature_cols = [col for col in ranker.feature_names_]
        for col in feature_cols:
            if col not in ranking_df.columns:
                ranking_df[col] = 0  # Fill missing columns with default value
        
        # Step 4: Predict scores using the ranking model
        ranking_scores = ranker.predict_proba(ranking_df[feature_cols])[:, 1]
        
        # Step 5: Sort and return recommendations
        recommendations = candidate_events.copy()
        recommendations['score'] = ranking_scores
        recommendations = recommendations.sort_values('score', ascending=False).head(final_k)
        
        return recommendations
    
    # Test the recommendation pipeline
    test_user_id = users_df['user_id'].iloc[0]
    recommendations = recommend_events_for_user(test_user_id)
    
    print(f"\nTop recommendations for user {test_user_id}:")
    print(recommendations[['title', 'event_category', 'event_city', 'weather_condition', 'temperature', 'score']])
    
    return recommend_events_for_user

recommendation_function = create_recommendation_pipeline()

# Function to explain recommendations based on weather and other factors
def explain_recommendations(user_id, recommendations):
    user = users_df[users_df['user_id'] == user_id].iloc[0]
    
    print(f"\nExplanation of recommendations for user {user_id}:")
    print(f"User Profile: {user['gender']}, {user['age']} years old, from {user['user_city']}")
    print(f"Weather Preference: {user['user_weather_preference']}")
    print(f"Interests: {user['user_interests']}")
    
    for i, (_, rec) in enumerate(recommendations.iterrows(), 1):
        print(f"\n{i}. {rec['title']}")
        print(f"   Category: {rec['event_category']}")
        print(f"   Weather: {rec['weather_condition']} at {rec['temperature']}°C")
        print(f"   Location: {rec['event_city']}")
        
        # Explain why this was recommended
        reasons = []
        
        # Check if location matches
        if rec['event_city'] == user['user_city']:
            reasons.append("📍 This event is in your city")
        
        # Check if category matches interests
        user_interests = user['user_interests'].split(',')
        for interest in user_interests:
            if interest.lower() in rec['event_category'].lower():
                reasons.append(f"🎯 Matches your interest in {interest}")
                break
        
        # Check weather preference match
        if user['user_weather_preference'] == 'indoor' and rec['event_indoor_capability']:
            reasons.append("🏠 This is an indoor event matching your preference")
        elif user['user_weather_preference'] == 'outdoor' and not rec['event_indoor_capability']:
            reasons.append("🌳 This is an outdoor event matching your preference")
        elif rec['weather_condition'] == 'Clear':
            reasons.append("☀️ The weather is expected to be clear")
        
        # Check temperature
        if 18 <= rec['temperature'] <= 24:
            reasons.append("🌡️ The temperature is in the comfortable range")
        
        print("   Why we recommended this:", " ".join(reasons))

# Test explanation with a sample user
test_user_id = users_df['user_id'].iloc[0]
recommendations = recommend_events_for_user(test_user_id)
explain_recommendations(test_user_id, recommendations)